In [3]:
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
import time

In [2]:
link ='https://www.crvownersclub.com/forums/problems-issues.14/page-2'
response=get(link)
html_soup = BeautifulSoup(response.text, 'html.parser')
posts = html_soup.find_all('div', class_="california-thread-item")
print(type(posts), len(posts))

<class 'bs4.element.ResultSet'> 35


In [194]:
post1 = posts[0]

In [30]:
title = post1.a.text
reply_count = post1.find('div', class_= 'reply-count').text.strip()
view_count = post1.find('div', class_='structItem-cell forum-view-stats view-count-cell').text.strip()
#post_time = 

In [43]:
user = post1.find('a', class_="username")
user_id = user['data-user-id']
user_name = user.text

In [187]:
user_join_date = post1.find('li', class_="structItem-startDate").find('a').find('time')['datetime']

In [192]:
title_head = post1.find('div', class_= "structItem-title") 
title=title_head.text.strip()
post_url = 'https://www.crvownersclub.com' + title_head.find('a')['href']

In [195]:
def page_posts(page_url):
    response = get(page_url)
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('div', class_="california-thread-item")
    return posts

In [161]:
response = get(link)
html_soup = BeautifulSoup(response.text, 'html.parser')
posts = html_soup.find_all('div', class_="bbWrapper")
reply_count = html_soup.find('span', class_='count').text
pages = int(reply_count)//20 +1
for i in range(pages-1):
    link1 = link + 'page-%d' %(i+2)
    print(link1)
print(type(posts), len(posts), stat, pages)

def thread_posts(thread_url):
    response = get(thread_url)
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('div', class_="bbWrapper")
    reply_count = html_soup.find('span', class_='count').text
    if 'K' in reply_count:
        reply_count = reply_count[:-1]+'000'
    pages = int(reply_count)//20 +1
    if pages>1:
        for i in range(pages-1):
            link2 = thread_url + 'page-%d' %(i+2)
            response = get(link2)
            html_soup = BeautifulSoup(response.text, 'html.parser')
            posts += html_soup.find_all('div', class_="bbWrapper")
    return posts

def filer_thread_post(posts):
    docs = []
    for i in range(len(posts)):
        text = posts[i].text
        tok = 'Click to expand...'
        if tok in text:
            index =  text.find(tok)
            text = text[index+len(tok)+1:]
        text = text.replace('\n',' ').replace('...',' ').replace('..',' ')
        if len(text)*' ' != text:
            docs.append(text.replace('\n',' ').replace('...',' ').replace('..',' '))
    return docs

def get_thread_docs(thread_url):
    posts = thread_posts(thread_url)
    docs = filer_thread_post(posts)
    return docs

https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/page-2
https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/page-3
https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/page-4
<class 'bs4.element.ResultSet'> 21 76 4


In [217]:
def main_post_info(post):
    reply_count = post.find('div', class_= 'reply-count').text.strip()
    view_count = post.find('div', class_='structItem-cell forum-view-stats view-count-cell').text.strip()
    user = post.find('a', class_="username")
    user_id = user['data-user-id']
    user_name = user.text
    user_join_date = post.find('li', class_="structItem-startDate").find('a').find('time')['datetime']

    title_head = post.find('div', class_= "structItem-title") 
    post_title = title_head.text.strip()
    post_url = 'https://www.crvownersclub.com' + title_head.find('a')['href']
    return user_id, user_name, user_join_date, post_title, reply_count, view_count, post_url 

In [218]:
main_post_info(post1)

('94961',
 'noliver87',
 '2014-05-20T09:26:34-0400',
 '2002 Honda CR-V RPM Gauge Malfunctioning.. Any Solutions or Ideas?',
 '5',
 '5K',
 'https://www.crvownersclub.com/threads/2002-honda-cr-v-rpm-gauge-malfunctioning-any-solutions-or-ideas.49561/')

In [198]:
print('title%d' %1)

title1


In [255]:
t0 = time.time()
post_title = []
post_url = []
reply_count = []
view_count = []
join_date = []
user_name = []
user_id = []
for i in range(360):
    page_i_url ='https://www.crvownersclub.com/forums/problems-issues.14/page-%d' %i
    page_i_posts = page_posts(page_i_url)
    for post in page_i_posts:
        post_info = main_post_info(post)
        uid, uname, ujoin_date, title, reply, view, url = post_info
        if title[:7]=='Sticky\n':
            title = title[8:]
        post_title.append(title)
        post_url.append(url)
        reply_count.append(reply)
        view_count.append(view)
        join_date.append(ujoin_date)
        user_name.append(uname)
        user_id.append(uid)
            
print(time.time()-t0)

258.9506561756134


In [4]:
import pickle

In [256]:
df_posts_p360 = pd.DataFrame({'user name': user_name,
                         'user id': user_id,
                         'user join date': join_date,
                         'post title': post_title,
                         'post url': post_url,
                         'reply count': reply_count,
                         'view count': view_count})

In [5]:
#pickle.dump( df_posts_p360, open(  "df_posts_p360.p", "wb" ) )
df_posts_p360 = pickle.load( open( "df_posts_p360.p", "rb" ) )
df_posts_p10 = pickle.load( open( "df_posts_p10.p", "rb" ) )

In [6]:
df_posts_p10['post url'].iloc[:5]

0    https://www.crvownersclub.com/threads/2015-hon...
1    https://www.crvownersclub.com/threads/heater-c...
2    https://www.crvownersclub.com/threads/2014-tpm...
3    https://www.crvownersclub.com/threads/low-tire...
4    https://www.crvownersclub.com/threads/your-att...
Name: post url, dtype: object

In [8]:
df_posts = df_posts_p10
size = df_posts.size

In [167]:
all_docs = []
for i in range(size):
    #post_url = df_posts['post url'].iloc[i]
    #docs = get_thread_docs(post_url)
    all_docs.append(i)
    #print(i, '/', size, len(docs)) 

In [147]:
link = 'https://www.crvownersclub.com/threads/all-warning-lights-on-dashboard-lit-up-on-2017-crv.202109'
link = 'https://www.crvownersclub.com/threads/parasitic-draw-2010-crv.40777/page-3'
link = 'https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/'
response = get(link)
html_soup = BeautifulSoup(response.text, 'html.parser')
posts = html_soup.find_all('div', class_="bbWrapper")
reply_count = html_soup.find('span', class_='count').text
pages = int(reply_count)//20 +1
for i in range(pages-1):
    link1 = link + 'page-%d' %(i+2)
    print(link1)
print(type(posts), len(posts), stat, pages)

https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/page-2
https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/page-3
https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/page-4
<class 'bs4.element.ResultSet'> 21 76 4


In [157]:
thread_url = 'https://www.crvownersclub.com/threads/1999-rear-wheel-came-off-while-driving.201554/'
posts = thread_posts(thread_url)
docs = filer_thread_post(posts)

62

In [164]:
docs = []
for i in range(len(posts)):
    text = posts[i].text
    tok = 'Click to expand...'
    if tok in text:
        index =  text.find(tok)
        text = text[index+len(tok)+1:]
    docs.append(text.replace('\n',' ').replace('...',' ').replace('..',' '))
docs;